# Data Wrangling

This first capstone is intended to forecast the consumption of red meat in the United States over the next 10 years. We will look for patterns in potential social, economic, and environmental indicators that could be predictors of consumption. 

In this notebook, we will inspect and clean our datasets for this project. The way that FAO stores their data, they have two separate sheets for data before 2013 and after. Since we have the same number of columns in each, our first step will be to combine these into one.

In [118]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from lib.sb_utils import save_file

In [119]:
faostat = pd.read_csv('data/faostat.csv')
faostat.head()

,Domain,Area,Element,Item,Year,Value
0,Food Balances (2014-),United States of America,Production,Bovine Meat,2014,11698.0
1,Food Balances (2014-),United States of America,Production,Bovine Meat,2015,10778.0
2,Food Balances (2014-),United States of America,Production,Bovine Meat,2016,11471.0
3,Food Balances (2014-),United States of America,Production,Bovine Meat,2017,11907.0
4,Food Balances (2014-),United States of America,Production,Bovine Meat,2018,12219.0


In [120]:
faostat.dtypes

Domain      object
Area        object
Element     object
Item        object
Year         int64
Value      float64
dtype: object

In [121]:
faostat.Element.unique()

array(['Production', 'Import Quantity', 'Stock Variation',
       'Export Quantity', 'Losses', 'Processing', 'Other uses (non-food)',
       'Tourist consumption', 'Residuals'], dtype=object)

In [122]:
faostat.Item.unique()

array(['Bovine Meat', 'Mutton & Goat Meat', 'Pigmeat', 'Poultry Meat',
       'Meat, Other'], dtype=object)

In [123]:
faostat['Domain'].unique()

array(['Food Balances (2014-)',
       'Food Balances (-2013, old methodology and population)'],
      dtype=object)

In [124]:
faostat[(faostat.Item=='Pigmeat') & (faostat.Year==2008)]

,Domain,Area,Element,Item,Year,Value
1226,"Food Balances (-2013, old methodology and popu...",United States of America,Production,Pigmeat,2008,10599.0
1279,"Food Balances (-2013, old methodology and popu...",United States of America,Import Quantity,Pigmeat,2008,454.0
1332,"Food Balances (-2013, old methodology and popu...",United States of America,Stock Variation,Pigmeat,2008,-19.0
1385,"Food Balances (-2013, old methodology and popu...",United States of America,Export Quantity,Pigmeat,2008,2129.0
1438,"Food Balances (-2013, old methodology and popu...",United States of America,Losses,Pigmeat,2008,NaN
1491,"Food Balances (-2013, old methodology and popu...",United States of America,Processing,Pigmeat,2008,0.0
1544,"Food Balances (-2013, old methodology and popu...",United States of America,Other uses (non-food),Pigmeat,2008,15.0
1597,"Food Balances (-2013, old methodology and popu...",United States of America,Tourist consumption,Pigmeat,2008,NaN
1650,"Food Balances (-2013, old methodology and popu...",United States of America,Residuals,Pigmeat,2008,NaN


Seems like we have some negative values. Let's get risk of those by multiplying by -1. We'll redo this later when we add and subtract elements from the items.

In [125]:
for i in range(len(faostat.Value.values)):
    if faostat.iloc[i,5] < 0:
        faostat.iloc[i,5]*=-1

In [126]:
faostat.Value.min()

0.0

## Null Values
Next we'll check for null values. We can see from the output above, that the numeric data we care most about is the Value column. We'll want to check if and when values are null and if that corresponds to null columns.

In [127]:
faostat.isna().sum()

Domain        0
Area          0
Element       0
Item          0
Year          0
Value      1077
dtype: int64

We only have null values in our Value column, we should fill those values with 0. This will preserve the observation.

In [128]:
faostat = faostat.fillna(0)

It looks like the "Domain" column is not relevant to our analysis - it only speaks to what methodology the balance sheet used - so we'll drop it. We'll also drop the Area since it's the same for all our data.

In [129]:
faostat.drop(labels=['Domain', 'Area'], axis=1, inplace=True)

## Understanding "Elements"
We don't have an precise measure for food consumption yet. We have a variety of "Elements" whose values can be taken together to measure how much meat was consumed. We don't have a sense of what the range of values here is and what makes sense to subtract and add together. Let's see what the mean value for each element can tell us.

In [130]:
faostat.groupby('Element')['Value'].mean()

Element
Export Quantity           546.572414
Import Quantity           305.789655
Losses                      0.000000
Other uses (non-food)      36.610345
Processing                  1.265517
Production               6111.700000
Residuals                   1.693103
Stock Variation            15.568966
Tourist consumption         0.000000
Name: Value, dtype: float64

In [131]:
faostat=faostat[faostat.Element!='Losses']

In [132]:
faostat=faostat[faostat.Element!='Tourist consumption']

In [133]:
faostat.groupby('Element')['Value'].mean()

Element
Export Quantity           546.572414
Import Quantity           305.789655
Other uses (non-food)      36.610345
Processing                  1.265517
Production               6111.700000
Residuals                   1.693103
Stock Variation            15.568966
Name: Value, dtype: float64

A few interesting things stand out: Production is by far the highest. That makes sense because the United States is a big producer of meat. That goes hand in hand with being a strong exporter of meat, which is the next highest mean. The rest of our values are a lot lower by comparison. Through some googling we have learned that Processing, Other uses, Stock variation, and Residuals all *take away* from the overall production value. These are associated with food being lost before it reaches a consumer, so we'll want to subtract them from the overall production value. 

## Create Consumption Dataframe
Now we will copy the faostat dataframe and multiply all element values which take away from the overall consumption values by -1. This will let us sum all element values for each meat type which will give us an overall consumption values.

In [134]:
#Multiple all element values by -1 where they should be subtracted
consumption = faostat.copy()
pos_elems = ['Import Quantity', 'Production']
neg_elems = set(list(consumption.Element.unique())) - set(pos_elems)
for neg in neg_elems:
    consumption.loc[consumption.Element==neg, ['Value']] *=-1

In [135]:
#clean consumption df
consumption.reset_index(inplace=True)
consumption.drop(labels='index', axis=1, inplace=True)
consumption.rename(columns={'Value':'Consumption'}, inplace=True)
consumption.head()

,Element,Item,Year,Consumption
0,Production,Bovine Meat,2014,11698.0
1,Production,Bovine Meat,2015,10778.0
2,Production,Bovine Meat,2016,11471.0
3,Production,Bovine Meat,2017,11907.0
4,Production,Bovine Meat,2018,12219.0


Next we need to group by Item and Year so that we can get the sum total of consumption for each type of meat (the positive elements - the elements that take away from the consumption value).

In [136]:
#Filter dataframe by item and year to get relevant consumption value
consumption_per_item = consumption.groupby(['Item','Year']).sum().reset_index()
consumption_per_item.head()

,Item,Year,Consumption
0,Bovine Meat,1961,7802.0
1,Bovine Meat,1962,7973.0
2,Bovine Meat,1963,8506.0
3,Bovine Meat,1964,9264.0
4,Bovine Meat,1965,9332.0


In [160]:
#If we want to see average values for each item per year
all_avg = consumption.groupby(['Year']).mean().reset_index()
all_avg[:5]

,Year,Consumption
0,1961,479.971429
1,1962,489.000000
2,1963,516.057143
3,1964,540.742857
4,1965,532.342857


## Population Data
We will also import data for total population, consumer price for agriculture, and employment in agriculture. These indicators may provide context to understand why meat consumption increases or decreases over time.

In [138]:
pop = pd.read_csv('data/pop.csv')
pop.head()

,Element,Year,Value
0,Rural population,1961,56180.595
1,Urban population,1961,133474.310
2,Rural population,1962,56262.705
3,Urban population,1962,136135.036
4,Rural population,1963,56293.690


In [139]:
pop.rename(columns={'Value':'Pop'}, inplace=True)

In [140]:
pop.isna().sum(), pop.shape

(Element    0
 Year       0
 Pop        0
 dtype: int64,
 (116, 3))

In [141]:
pop = pop.pivot(index ='Year', columns='Element', values='Pop')
pop.head()

Element,Rural population,Urban population
Year,,
1961,56180.595,133474.310
1962,56262.705,136135.036
1963,56293.690,138722.521
1964,56268.964,141222.880
1965,56190.881,143624.659


## Price

In [154]:
price = pd.read_csv('data/prices.csv')
price.head()

,Element,Item,Year,Unit,Value
0,Import Quantity,Bovine Meat,1961,tonnes,387520
1,Import Quantity,Bovine Meat,1962,tonnes,554262
2,Import Quantity,Bovine Meat,1963,tonnes,647256
3,Import Quantity,Bovine Meat,1964,tonnes,468543
4,Import Quantity,Bovine Meat,1965,tonnes,410271


In [167]:
price.Unit.unique(), price.Item.unique(), price.Element.unique()

(array(['tonnes', '1000 US$'], dtype=object),
 array(['Bovine Meat', 'Poultry Meat', 'Pigmeat', 'Other Meat'],
       dtype=object),
 array(['Import Quantity', 'Import Value', 'Export Quantity',
        'Export Value'], dtype=object))

In [156]:
price.isna().sum(), price.shape, 

(Element    0
 Item       0
 Year       0
 Unit       0
 Value      0
 dtype: int64,
 (944, 5))

It would be easier if we just had one price per import/export quantity as opposed to separate values. To do this, we'll multiple dollar Values by 1000 (their unit) and divide by the import/export quantity.

In [161]:
for i in range(len(price.Value)):
    if price.iloc[i,3]=='1000 US$':
        price.iloc[i,4]*=1000

In [172]:
temp = price[(price.Item == 'Bovine Meat') & (price.Year==1961)]
temp[temp.Element == 'Import Value'].Value.values/temp[temp.Element == 'Import Quantity'].Value.values

array([590.31017754])

## Employment Data
Next we'll look at our employment data, see what columns can be dropped, whether there are null values, and finally find an average price per year.

In [32]:
employment = pd.read_csv('data/employment.csv')
employment[employment.Value.isna()==False]

,Domain,Area,Indicator,Source,Year,Value
88,Employment Indicators,United States of America,"Agriculture value added per worker (US$, 2010 ...",Labour force survey,2008,69986.273438
89,Employment Indicators,United States of America,Employment in agriculture,Labour force survey,2008,1943.790000
90,Employment Indicators,United States of America,"Agriculture value added per worker (US$, 2010 ...",Labour force survey,2009,79915.085938
91,Employment Indicators,United States of America,Employment in agriculture,Labour force survey,2009,1888.286000
92,Employment Indicators,United States of America,"Agriculture value added per worker (US$, 2010 ...",Labour force survey,2010,73930.257813
93,Employment Indicators,United States of America,Employment in agriculture,Labour force survey,2010,1978.892000
94,Employment Indicators,United States of America,"Agriculture value added per worker (US$, 2010 ...",Labour force survey,2011,69596.734375
95,Employment Indicators,United States of America,Employment in agriculture,Labour force survey,2011,2020.801000
96,Employment Indicators,United States of America,"Agriculture value added per worker (US$, 2010 ...",Labour force survey,2012,68901.375000
97,Employment Indicators,United States of America,Employment in agriculture,Labour force survey,2012,1966.731000


In [36]:
employment[employment.Source !='Labour force survey']

,Domain,Area,Indicator,Source,Year,Value
0,Employment Indicators,United States of America,"Agriculture value added per worker (US$, 2010 ...",Administrative records,2008,NaN
1,Employment Indicators,United States of America,Employment in agriculture,Administrative records,2008,NaN
2,Employment Indicators,United States of America,"Agriculture value added per worker (US$, 2010 ...",Administrative records,2009,NaN
3,Employment Indicators,United States of America,Employment in agriculture,Administrative records,2009,NaN
4,Employment Indicators,United States of America,"Agriculture value added per worker (US$, 2010 ...",Administrative records,2010,NaN
...,...,...,...,...,...,...
149,Employment Indicators,United States of America,Employment in agriculture,Population census,2016,NaN
150,Employment Indicators,United States of America,"Agriculture value added per worker (US$, 2010 ...",Population census,2017,NaN
151,Employment Indicators,United States of America,Employment in agriculture,Population census,2017,NaN
152,Employment Indicators,United States of America,"Agriculture value added per worker (US$, 2010 ...",Population census,2018,NaN


In [32]:
employment.Value.isna().sum(), employment.shape

(133, (154, 6))

In [33]:
employment.Indicator.unique(), employment.Source.unique()

(array(['Agriculture value added per worker (US$, 2010 prices)',
        'Employment in agriculture'], dtype=object),
 array(['Administrative records', 'Employment surveys',
        'Household income and expenditure survey', 'Household survey',
        'Labour force survey', 'Official estimates', 'Population census'],
       dtype=object))

In [34]:
employment = employment[~employment.Value.isna()]
employment = employment[employment.Indicator=='Employment in agriculture']

In [35]:
employment.drop(labels=['Domain', 'Area', 'Source', 'Indicator'], axis=1,inplace=True)
employment.rename(columns={'Value':'Employment'}, inplace=True)

In [36]:
employment.reset_index(inplace=True)
employment.drop(labels='index', axis=1, inplace=True)
employment.head()

,Year,Employment
0,2008,1943.790
1,2009,1888.286
2,2010,1978.892
3,2011,2020.801
4,2012,1966.731


## Combining Tables

In [59]:
merged1 = pd.merge(consumption_per_item, pop, how='left', on='Year')
merged1.head()

,Item,Year,Consumption,Pop
0,Bovine Meat,2008,12444.0,303486.012
1,Bovine Meat,2009,12258.0,306307.567
2,Bovine Meat,2010,12072.0,309011.475
3,Bovine Meat,2011,11592.0,311584.047
4,Bovine Meat,2012,11725.0,314043.885


In [60]:
merged2 = pd.merge(merged1, price_per_year, how='left', on='Year')
merged2.head()

,Item,Year,Consumption,Pop,Price
0,Bovine Meat,2008,12444.0,303486.012,89.511370
1,Bovine Meat,2009,12258.0,306307.567,89.552337
2,Bovine Meat,2010,12072.0,309011.475,90.440451
3,Bovine Meat,2011,11592.0,311584.047,94.027484
4,Bovine Meat,2012,11725.0,314043.885,96.152371


In [61]:
merged_indicators = pd.merge(merged2, employment, how='left', on='Year')
merged_indicators.head()

,Item,Year,Consumption,Pop,Price,Employment
0,Bovine Meat,2008,12444.0,303486.012,89.511370,1943.790
1,Bovine Meat,2009,12258.0,306307.567,89.552337,1888.286
2,Bovine Meat,2010,12072.0,309011.475,90.440451,1978.892
3,Bovine Meat,2011,11592.0,311584.047,94.027484,2020.801
4,Bovine Meat,2012,11725.0,314043.885,96.152371,1966.731


In [65]:
datapath = 'data' 
save_file(merged_indicators, 'merged_data.csv', datapath)

Writing file.  "data/merged_data.csv"
